# Junyi Learning Analytics

---

This project aims to analyse online learning activities from Junyi Academy Foundation.

### Data Source

[Kaggle](https://www.kaggle.com/datasets/junyiacademy/learning-activity-public-dataset-by-junyi-academy), made available under [CC-BY-NC-SA-4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/).

### Analysis Tool

We will be using PostgreSQL for our data analysis.

## Exploration: Pandas

In [4]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

import matplotlib.pyplot as plt
import seaborn as sns

log_data = pd.read_csv("junyi_data/Log_Problem.csv", nrows=10000, skiprows=(2, 10000))

In [5]:
profile = ProfileReport(log_data, title="Profiling Report")
profile.to_file("report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]